In [4]:
import sys

!pip install numpy pandas sentence-transformers umap-learn plotly scikit-learn
print(f"Python version: {sys.version}")
print("✅ Pakker installert!")

  Using cached numpy-2.3.5-cp314-cp314-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pandas-2.3.3-cp314-cp314-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached pandas-2.3.3-cp314-cp314-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached sentence_transformers-5.1.2-py3-none-any.whl.metadata (16 kB)
  Using cached sentence_transformers-5.1.2-py3-none-any.whl.metadata (16 kB)
  Using cached umap_learn-0.5.9.post2-py3-none-any.whl.metadata (25 kB)
  Using cached umap_learn-0.5.9.post2-py3-none-any.whl.metadata (25 kB)
  Using cached plotly-6.5.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached plotly-6.5.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached scikit_learn-1.7.2-cp314-cp314-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scikit_learn-1.7.2-cp314-cp314-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached pytz-2025.2-py2.py3-no

# 🧠 Embedding-modeller for RAG

**Mål:** Forstå hvordan valg av embedding-modell påvirker retrieval-kvaliteten i et RAG-system.

## Hva er en embedding?

En embedding er en **numerisk representasjon** av tekst som fanger opp semantisk mening. 
Tekst som betyr lignende ting vil ha vektorer som ligger nær hverandre i vektorrommet.

```
"Hva er dokumentavgift?" → [0.12, -0.34, 0.56, ..., 0.78]  (768 dimensjoner)
"Dokumentavgift forklart" → [0.11, -0.32, 0.55, ..., 0.79]  (lignende vektor!)
"Oppskrift på pizza"     → [0.89, 0.23, -0.45, ..., -0.12] (helt annen vektor)
```

## Hvorfor er valg av modell viktig?

| Aspekt | Påvirkning |
|--------|------------|
| **Dimensjonalitet** | Høyere = mer presis, men tregere og mer lagring |
| **Språkstøtte** | Kritisk for norske dokumenter! |
| **Treningsdata** | Modeller trent på juridisk tekst forstår juss bedre |
| **Modellstørrelse** | Større modeller er ofte bedre, men tregere |

## 📦 Installer pakker

In [7]:
# Kjør dette først for å installere nødvendige pakker
# UMAP krever Python <3.14, så vi bruker PCA som alternativ
!pip install sentence-transformers plotly pandas numpy scikit-learn

  Using cached sentence_transformers-5.1.2-py3-none-any.whl.metadata (16 kB)
  Using cached umap_learn-0.5.9.post2-py3-none-any.whl.metadata (25 kB)
  Using cached umap_learn-0.5.9.post2-py3-none-any.whl.metadata (25 kB)
  Using cached plotly-6.5.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached plotly-6.5.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached pandas-2.3.3-cp314-cp314-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached pandas-2.3.3-cp314-cp314-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached numpy-2.3.5-cp314-cp314-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached numpy-2.3.5-cp314-cp314-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached scikit_learn-1.7.2-cp314-cp314-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scikit_learn-1.7.2-cp314-cp314-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached tqdm-4.67.1-py3-

In [8]:
import json
import time
from pathlib import Path
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'numpy'

## 📄 Last inn chunks fra chunking-steget

Vi bruker chunks som allerede er laget av chunking-notebooken.

In [ ]:
# Last inn metadata chunks (beste kvalitet fra chunking-steget)
chunks_path = Path("../output/chunks_metadata.jsonl")

chunks = []
with open(chunks_path, "r", encoding="utf-8") as f:
    for line in f:
        chunks.append(json.loads(line))

print(f"📊 Lastet {len(chunks)} chunks")
print(f"\n📝 Eksempel chunk:")
print(f"   Text: {chunks[20]['text'][:100]}...")
print(f"   Context: {chunks[20].get('context', 'N/A')}")

## 🤖 Definer embedding-modeller

Vi sammenligner **tre forskjellige open source modeller** med ulike egenskaper:

| Modell | Dimensjoner | Språk | Størrelse | Bruksområde |
|--------|-------------|-------|-----------|-------------|
| `all-MiniLM-L6-v2` | 384 | Engelsk | 80 MB | Rask, generell |
| `paraphrase-multilingual-MiniLM-L12-v2` | 384 | 50+ språk | 420 MB | Flerspråklig |
| `multilingual-e5-large` | 1024 | 100 språk | 2.2 GB | State-of-the-art |

In [ ]:
# Definer modellene vi skal teste
MODELS = {
    "MiniLM-EN": {
        "name": "all-MiniLM-L6-v2",
        "dim": 384,
        "desc": "Rask engelsk modell",
        "color": "#FF6B6B"
    },
    "MiniLM-Multi": {
        "name": "paraphrase-multilingual-MiniLM-L12-v2", 
        "dim": 384,
        "desc": "Flerspråklig (50+ språk)",
        "color": "#4ECDC4"
    },
    "E5-Large": {
        "name": "intfloat/multilingual-e5-large",
        "dim": 1024,
        "desc": "State-of-the-art multilingual",
        "color": "#45B7D1"
    }
}

print("🤖 Modeller som skal testes:")
for key, info in MODELS.items():
    print(f"   • {key}: {info['desc']} ({info['dim']} dim)")

In [ ]:
# Last inn modellene (kan ta litt tid første gang)
models = {}

for key, info in MODELS.items():
    print(f"⏳ Laster {key}...")
    start = time.time()
    models[key] = SentenceTransformer(info["name"])
    elapsed = time.time() - start
    print(f"   ✅ Lastet på {elapsed:.1f}s")

print("\n🎉 Alle modeller lastet!")

## 🔬 Generer embeddings

Vi genererer embeddings for alle chunks med hver modell og måler tiden.

In [ ]:
# Hent ut tekst fra chunks
texts = [c["text"] for c in chunks]

# For E5-modellen må vi legge til "passage: " prefix
texts_e5 = [f"passage: {t}" for t in texts]

# Generer embeddings med hver modell
embeddings = {}
timing = {}

for key, model in models.items():
    print(f"\n⏳ Genererer embeddings med {key}...")
    
    # E5 krever "passage:" prefix for dokumenter
    input_texts = texts_e5 if "E5" in key else texts
    
    start = time.time()
    emb = model.encode(input_texts, show_progress_bar=True, normalize_embeddings=True)
    elapsed = time.time() - start
    
    embeddings[key] = emb
    timing[key] = elapsed
    
    print(f"   ✅ {len(texts)} chunks på {elapsed:.1f}s ({len(texts)/elapsed:.1f} chunks/sek)")
    print(f"   📐 Shape: {emb.shape}")

## 📊 Sammenlign ytelse

In [ ]:
# Lag sammenligning
comparison_data = []
for key, info in MODELS.items():
    emb = embeddings[key]
    comparison_data.append({
        "Modell": key,
        "Dimensjoner": info["dim"],
        "Tid (sek)": f"{timing[key]:.1f}",
        "Chunks/sek": f"{len(texts)/timing[key]:.1f}",
        "Minne (MB)": f"{emb.nbytes / 1024 / 1024:.1f}",
        "Beskrivelse": info["desc"]
    })

df_comparison = pd.DataFrame(comparison_data)
print("📊 Ytelsessammenligning:\n")
print(df_comparison.to_string(index=False))

In [ ]:
# Visualiser ytelse
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Tid per modell", "Dimensjoner vs Hastighet"),
    specs=[[{"type": "bar"}, {"type": "scatter"}]]
)

colors = [MODELS[k]["color"] for k in MODELS.keys()]

# Bar chart: Tid
fig.add_trace(
    go.Bar(
        x=list(MODELS.keys()),
        y=[timing[k] for k in MODELS.keys()],
        marker_color=colors,
        text=[f"{timing[k]:.1f}s" for k in MODELS.keys()],
        textposition="outside"
    ),
    row=1, col=1
)

# Scatter: Dimensjoner vs Hastighet
fig.add_trace(
    go.Scatter(
        x=[MODELS[k]["dim"] for k in MODELS.keys()],
        y=[len(texts)/timing[k] for k in MODELS.keys()],
        mode="markers+text",
        marker=dict(size=20, color=colors),
        text=list(MODELS.keys()),
        textposition="top center"
    ),
    row=1, col=2
)

fig.update_layout(
    title="⚡ Ytelsessammenligning av Embedding-modeller",
    showlegend=False,
    height=400
)
fig.update_xaxes(title_text="Modell", row=1, col=1)
fig.update_yaxes(title_text="Tid (sekunder)", row=1, col=1)
fig.update_xaxes(title_text="Dimensjoner", row=1, col=2)
fig.update_yaxes(title_text="Chunks per sekund", row=1, col=2)

fig.show()

## 🎨 Visualiser embedding-rom med PCA

PCA (Principal Component Analysis) lar oss visualisere høy-dimensjonale embeddings i 2D.

**Hva ser vi etter?**
- Chunks om samme tema skal klynge seg sammen
- Forskjellige modeller kan gruppere tekst ulikt

In [ ]:
from sklearn.decomposition import PCA

# Reduser dimensjoner med PCA for hver modell
pca_results = {}

for key in MODELS.keys():
    print(f"⏳ PCA for {key}...")
    reducer = PCA(n_components=2, random_state=42)
    pca_results[key] = reducer.fit_transform(embeddings[key])
    print(f"   ✅ Ferdig (forklart varians: {sum(reducer.explained_variance_ratio_):.1%})")

In [ ]:
# Hent metadata for fargelegging
# Bruk h1 header som kategori
categories = []
for c in chunks:
    meta = c.get("metadata", {})
    cat = meta.get("h3", meta.get("h2", meta.get("h1", "Ukjent")))
    # Forenkle kategorinavnet
    cat = cat.replace("**", "").strip()[:30]
    categories.append(cat)

# Finn de 8 vanligste kategoriene
from collections import Counter
cat_counts = Counter(categories)
top_cats = [c[0] for c in cat_counts.most_common(8)]
categories_simplified = [c if c in top_cats else "Andre" for c in categories]

print(f"📂 Kategorier funnet: {len(set(categories_simplified))}")

In [ ]:
# Lag PCA visualisering for alle modeller
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=[f"{k} ({MODELS[k]['dim']}d)" for k in MODELS.keys()]
)

for i, key in enumerate(MODELS.keys(), 1):
    pca_2d = pca_results[key]
    
    fig.add_trace(
        go.Scatter(
            x=pca_2d[:, 0],
            y=pca_2d[:, 1],
            mode="markers",
            marker=dict(
                size=6,
                opacity=0.7,
                color=[hash(c) % 10 for c in categories_simplified],
                colorscale="Viridis"
            ),
            text=[f"{c}\n{chunks[j]['text'][:50]}..." for j, c in enumerate(categories_simplified)],
            hovertemplate="%{text}<extra></extra>"
        ),
        row=1, col=i
    )

fig.update_layout(
    title="🎨 PCA: Hvordan ser modellene på dokumentet?",
    showlegend=False,
    height=500,
    width=1200
)

fig.show()

## 🔍 Test retrieval: Hvilken modell finner best svar?

Vi tester modellene med noen norske spørsmål om dokumentavgift.

In [ ]:
# Test-spørsmål på norsk
test_queries = [
    "Hva er dokumentavgift?",
    "Hvor mye er dokumentavgiften i prosent?",
    "Når slipper man å betale dokumentavgift?",
    "Hva skjer ved arv av eiendom?",
    "Hvordan beregnes avgiftsgrunnlaget?"
]

def search(query: str, model_key: str, top_k: int = 3) -> list:
    """Søk etter mest relevante chunks for en query"""
    model = models[model_key]
    
    # E5 krever "query:" prefix
    if "E5" in model_key:
        query = f"query: {query}"
    
    query_emb = model.encode([query], normalize_embeddings=True)
    
    # Beregn similarity
    similarities = cosine_similarity(query_emb, embeddings[model_key])[0]
    
    # Hent top-k
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    results = []
    for idx in top_indices:
        results.append({
            "chunk_id": chunks[idx]["chunk_id"],
            "score": float(similarities[idx]),
            "text": chunks[idx]["text"][:200],
            "context": chunks[idx].get("context", "")
        })
    return results

In [ ]:
# Test alle modeller med første query
query = test_queries[0]
print(f"🔍 Query: '{query}'\n")
print("=" * 80)

for model_key in MODELS.keys():
    print(f"\n📌 {model_key} ({MODELS[model_key]['desc']})")
    print("-" * 40)
    
    results = search(query, model_key, top_k=3)
    for i, r in enumerate(results, 1):
        print(f"  {i}. [Score: {r['score']:.3f}] {r['text'][:100]}...")

In [ ]:
# Sammenlign alle queries visuelt
all_results = []

for query in test_queries:
    for model_key in MODELS.keys():
        results = search(query, model_key, top_k=1)
        all_results.append({
            "Query": query[:40] + "...",
            "Modell": model_key,
            "Top Score": results[0]["score"],
            "Chunk ID": results[0]["chunk_id"]
        })

df_results = pd.DataFrame(all_results)

# Pivot for heatmap
pivot = df_results.pivot(index="Query", columns="Modell", values="Top Score")

fig = px.imshow(
    pivot,
    labels=dict(x="Modell", y="Spørsmål", color="Similarity Score"),
    color_continuous_scale="RdYlGn",
    aspect="auto",
    text_auto=".2f"
)

fig.update_layout(
    title="🎯 Retrieval Scores per Modell og Spørsmål",
    height=400
)

fig.show()

## 🇳🇴 Språktest: Hvordan håndterer modellene norsk?

Vi tester hvordan modellene matcher norske vs engelske queries mot norsk tekst.

In [ ]:
# Norsk vs Engelsk query
language_pairs = [
    ("Hva er dokumentavgift?", "What is document tax?"),
    ("Fritak ved arv", "Exemption for inheritance"),
    ("Beregning av avgift", "Calculation of tax")
]

print("🇳🇴 Norsk vs 🇬🇧 Engelsk queries på norsk dokument\n")
print("=" * 80)

for no_query, en_query in language_pairs:
    print(f"\n📝 NO: '{no_query}' | EN: '{en_query}'")
    print("-" * 60)
    
    for model_key in MODELS.keys():
        no_results = search(no_query, model_key, top_k=1)
        en_results = search(en_query, model_key, top_k=1)
        
        no_score = no_results[0]["score"]
        en_score = en_results[0]["score"]
        diff = no_score - en_score
        
        emoji = "✅" if diff > 0 else "⚠️"
        print(f"  {model_key:15} | NO: {no_score:.3f} | EN: {en_score:.3f} | Diff: {diff:+.3f} {emoji}")

## 📈 Intra-cluster similarity: Hvor konsistente er embeddings?

Vi måler hvor like chunks innenfor samme kategori er - høyere = bedre semantisk forståelse.

In [ ]:
from collections import defaultdict

# Grupper chunks per kategori
cat_to_indices = defaultdict(list)
for i, cat in enumerate(categories_simplified):
    if cat != "Andre":  # Skip "Andre"
        cat_to_indices[cat].append(i)

# Beregn gjennomsnittlig intra-cluster similarity
cluster_scores = []

for model_key in MODELS.keys():
    emb = embeddings[model_key]
    
    similarities = []
    for cat, indices in cat_to_indices.items():
        if len(indices) >= 2:
            cat_emb = emb[indices]
            sim_matrix = cosine_similarity(cat_emb)
            # Hent øvre triangel (unngå diagonalen)
            upper_tri = sim_matrix[np.triu_indices_from(sim_matrix, k=1)]
            similarities.extend(upper_tri)
    
    avg_sim = np.mean(similarities)
    cluster_scores.append({
        "Modell": model_key,
        "Avg Intra-Cluster Similarity": avg_sim
    })

df_cluster = pd.DataFrame(cluster_scores)

fig = px.bar(
    df_cluster,
    x="Modell",
    y="Avg Intra-Cluster Similarity",
    color="Modell",
    color_discrete_sequence=[MODELS[k]["color"] for k in MODELS.keys()],
    text_auto=".3f"
)

fig.update_layout(
    title="📊 Intra-Cluster Similarity (høyere = bedre gruppering)",
    showlegend=False,
    yaxis_range=[0, 1]
)

fig.show()

## 💾 Lagre embeddings for senere bruk

Vi lagrer embeddings slik at de kan brukes i retrieval-steget.

In [ ]:
output_dir = Path("./output")
output_dir.mkdir(exist_ok=True)

for model_key in MODELS.keys():
    # Lagre som numpy
    np.save(output_dir / f"embeddings_{model_key}.npy", embeddings[model_key])
    print(f"✅ Lagret embeddings_{model_key}.npy ({embeddings[model_key].shape})")

print(f"\n📂 Filer lagret i {output_dir.absolute()}")

## 🎓 Oppsummering

### Hva lærte vi?

1. **Språkstøtte er kritisk** - Engelske modeller sliter med norsk tekst
2. **Dimensjonalitet ≠ Kvalitet** - Flere dimensjoner betyr ikke alltid bedre resultater
3. **Trade-offs finnes** - Raskere modeller kan være "good enough" for mange use cases
4. **Test på dine data** - Benchmark-resultater gjelder ikke alltid for din use case

### Anbefalinger for norske RAG-systemer

| Scenario | Anbefalt modell |
|----------|----------------|
| Produksjon med norsk tekst | `multilingual-e5-large` |
| Rask prototyping | `paraphrase-multilingual-MiniLM-L12-v2` |
| Kun engelsk tekst | `all-MiniLM-L6-v2` |

### Neste steg

- **Hybridsøk** - Kombiner med BM25 for bedre resultater
- **Re-ranking** - Bruk cross-encoder for å forbedre ranking
- **Evaluering** - Mål MRR, Recall@k på reelle test-sett